In [21]:
import os
import cv2
import mediapipe as mp
import numpy as np
from tqdm import tqdm

# === INPUT & OUTPUT ===
INPUT_FOLDER = r"D:/DETECTION 1.0/Celeb-DF/Real Frames"
OUTPUT_FOLDER = r"D:/DETECTION 1.0/Celeb-DF/aligned_faces_real"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# === Initialize MediaPipe FaceMesh ===
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True)

# === Get eye landmarks for alignment ===
LEFT_EYE_INDEX = 33   # left eye outer corner
RIGHT_EYE_INDEX = 263 # right eye outer corner

def align_face(image, landmarks):
    h, w = image.shape[:2]

    # Get eye coordinates
    left_eye = landmarks[LEFT_EYE_INDEX]
    right_eye = landmarks[RIGHT_EYE_INDEX]
    lx, ly = int(left_eye.x * w), int(left_eye.y * h)
    rx, ry = int(right_eye.x * w), int(right_eye.y * h)

    # Compute angle
    dx = rx - lx
    dy = ry - ly
    angle = np.degrees(np.arctan2(dy, dx))

    # Get center point between eyes
    center = ((lx + rx) // 2, (ly + ry) // 2)

    # Rotate image to align eyes horizontally
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    aligned = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_LINEAR)
    return aligned

def process_video_folder(video_folder, video_id):
    output_path = os.path.join(OUTPUT_FOLDER, video_id)
    os.makedirs(output_path, exist_ok=True)

    for img_file in sorted(os.listdir(video_folder)):
        if not img_file.lower().endswith(('.jpg', '.png')):
            continue

        img_path = os.path.join(video_folder, img_file)
        image = cv2.imread(img_path)
        if image is None:
            continue

        # Convert to RGB
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        result = face_mesh.process(rgb)

        if result.multi_face_landmarks:
            landmarks = result.multi_face_landmarks[0].landmark
            aligned_img = align_face(image, landmarks)

            # Save aligned image
            cv2.imwrite(os.path.join(output_path, img_file), aligned_img)

def align_all_videos(input_root):
    video_ids = sorted(os.listdir(input_root))
    for vid in tqdm(video_ids, desc="🔄 Aligning videos"):
        video_folder = os.path.join(input_root, vid)
        if os.path.isdir(video_folder):
            process_video_folder(video_folder, vid)

# === Run alignment ===
align_all_videos(INPUT_FOLDER)
print(f"\n✅ All aligned faces saved to: {OUTPUT_FOLDER}")


🔄 Aligning videos: 100%|██████████| 408/408 [1:05:03<00:00,  9.57s/it]


✅ All aligned faces saved to: D:/DETECTION 1.0/Celeb-DF/aligned_faces_real


In [22]:
import os
import cv2
import mediapipe as mp
import numpy as np
from tqdm import tqdm

# === INPUT & OUTPUT ===
INPUT_FOLDER = r"D:\DETECTION 1.0\Celeb-DF\Fake Frames"
OUTPUT_FOLDER = r"D:/DETECTION 1.0/Celeb-DF/aligned_faces_fake"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# === Initialize MediaPipe FaceMesh ===
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True)

# === Get eye landmarks for alignment ===
LEFT_EYE_INDEX = 33   # left eye outer corner
RIGHT_EYE_INDEX = 263 # right eye outer corner

def align_face(image, landmarks):
    h, w = image.shape[:2]

    # Get eye coordinates
    left_eye = landmarks[LEFT_EYE_INDEX]
    right_eye = landmarks[RIGHT_EYE_INDEX]
    lx, ly = int(left_eye.x * w), int(left_eye.y * h)
    rx, ry = int(right_eye.x * w), int(right_eye.y * h)

    # Compute angle
    dx = rx - lx
    dy = ry - ly
    angle = np.degrees(np.arctan2(dy, dx))

    # Get center point between eyes
    center = ((lx + rx) // 2, (ly + ry) // 2)

    # Rotate image to align eyes horizontally
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    aligned = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_LINEAR)
    return aligned

def process_video_folder(video_folder, video_id):
    output_path = os.path.join(OUTPUT_FOLDER, video_id)
    os.makedirs(output_path, exist_ok=True)

    for img_file in sorted(os.listdir(video_folder)):
        if not img_file.lower().endswith(('.jpg', '.png')):
            continue

        img_path = os.path.join(video_folder, img_file)
        image = cv2.imread(img_path)
        if image is None:
            continue

        # Convert to RGB
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        result = face_mesh.process(rgb)

        if result.multi_face_landmarks:
            landmarks = result.multi_face_landmarks[0].landmark
            aligned_img = align_face(image, landmarks)

            # Save aligned image
            cv2.imwrite(os.path.join(output_path, img_file), aligned_img)

def align_all_videos(input_root):
    video_ids = sorted(os.listdir(input_root))
    for vid in tqdm(video_ids, desc="🔄 Aligning videos"):
        video_folder = os.path.join(input_root, vid)
        if os.path.isdir(video_folder):
            process_video_folder(video_folder, vid)

# === Run alignment ===
align_all_videos(INPUT_FOLDER)
print(f"\n✅ All aligned faces saved to: {OUTPUT_FOLDER}")


🔄 Aligning videos: 100%|██████████| 408/408 [1:00:22<00:00,  8.88s/it]


✅ All aligned faces saved to: D:/DETECTION 1.0/Celeb-DF/aligned_faces_fake


In [23]:
import os
import cv2
import numpy as np
import pandas as pd
import mediapipe as mp
from tqdm import tqdm
from scipy.spatial import distance

# === Setup ===
INPUT_FOLDER = r"D:\DETECTION 1.0\Celeb-DF\aligned_faces_fake"# Folder containing aligned face frames per video
OUTPUT_FOLDER = r"D:\DETECTION 1.0\Celeb-DF\muscle_tension_maps"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# === MediaPipe Face Mesh ===
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True)

# === Adjacency Pairs (subset for strain) ===
ADJACENT_PAIRS = [
    (33, 160), (160, 158), (158, 133),  # Left Eye Top
    (263, 387), (387, 385), (385, 362),  # Right Eye Top
    (61, 291),  # Mouth
    (50, 280),  # Cheeks
    (70, 336),  # Brows
    (10, 152),  # Nose Bridge to Chin
]

# === Compute Strain Between Landmark Pairs ===
def compute_strain(landmarks, image_shape):
    h, w = image_shape[:2]
    coords = [(int(lm.x * w), int(lm.y * h)) for lm in landmarks]
    strains = []
    for (i, j) in ADJACENT_PAIRS:
        if i < len(coords) and j < len(coords):
            dist = distance.euclidean(coords[i], coords[j])
            strains.append(((coords[i], coords[j]), dist))
    return strains

# === Draw Tension Map ===
def draw_tension_map(image, strain_data):
    tension_map = np.zeros_like(image)
    for ((pt1, pt2), val) in strain_data:
        intensity = int(min(max(val, 0), 50)) * 5  # Scale
        cv2.line(tension_map, pt1, pt2, (0, 0, intensity), 2)
    heatmap = cv2.applyColorMap(tension_map[:, :, 2], cv2.COLORMAP_JET)
    blended = cv2.addWeighted(image, 0.6, heatmap, 0.4, 0)
    return blended

# === Process Folder ===
def process_video_folder(video_path, output_subfolder):
    os.makedirs(output_subfolder, exist_ok=True)
    for img_file in tqdm(sorted(os.listdir(video_path)), desc=f"Processing {os.path.basename(video_path)}"):
        if not img_file.endswith((".jpg", ".png")):
            continue
        img_path = os.path.join(video_path, img_file)
        image = cv2.imread(img_path)
        if image is None:
            continue

        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb)
        if results.multi_face_landmarks:
            lm = results.multi_face_landmarks[0].landmark
            strain_data = compute_strain(lm, image.shape)
            tension_map = draw_tension_map(image, strain_data)
            save_path = os.path.join(output_subfolder, img_file)
            cv2.imwrite(save_path, tension_map)

# === Run on All Videos ===
for video_id in os.listdir(INPUT_FOLDER):
    video_folder = os.path.join(INPUT_FOLDER, video_id)
    if os.path.isdir(video_folder):
        output_subfolder = os.path.join(OUTPUT_FOLDER, video_id)
        process_video_folder(video_folder, output_subfolder)

print("✅ Muscle tension maps saved to:", OUTPUT_FOLDER)


Processing id8_id1_0003: 100%|██████████| 388/388 [00:09<00:00, 42.54it/s]
Processing id8_id1_0007: 0it [00:00, ?it/s]
Processing id8_id5_0003: 100%|██████████| 384/384 [00:08<00:00, 44.33it/s]
Processing id8_id5_0007: 0it [00:00, ?it/s]
Processing id8_id5_0008: 0it [00:00, ?it/s]
Processing id9_id6_0009: 100%|██████████| 313/313 [00:08<00:00, 36.72it/s]

✅ Muscle tension maps saved to: D:\DETECTION 1.0\Celeb-DF\muscle_tension_maps


In [1]:
import os
import cv2
import numpy as np
import mediapipe as mp
from tqdm import tqdm
from scipy.spatial import distance

# === INPUT CONFIG ===
REAL_INPUT = r"D:\DETECTION 1.0\Celeb-DF\aligned_faces_real"
FAKE_INPUT = r"D:\DETECTION 1.0\Celeb-DF\aligned_faces_fake"
OUTPUT_FOLDER = r"D:/DETECTION 1.0/Celeb-DF/muscle_tension_maps"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# === MediaPipe Face Mesh ===
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True)

# === Adjacency Pairs (subset for strain) ===
ADJACENT_PAIRS = [
    (33, 160), (160, 158), (158, 133),  # Left Eye Top
    (263, 387), (387, 385), (385, 362),  # Right Eye Top
    (61, 291),  # Mouth
    (50, 280),  # Cheeks
    (70, 336),  # Brows
    (10, 152),  # Nose Bridge to Chin
]

# === Compute Strain Between Landmark Pairs ===
def compute_strain(landmarks, image_shape):
    h, w = image_shape[:2]
    coords = [(int(lm.x * w), int(lm.y * h)) for lm in landmarks]
    strains = []
    for (i, j) in ADJACENT_PAIRS:
        if i < len(coords) and j < len(coords):
            dist = distance.euclidean(coords[i], coords[j])
            strains.append(((coords[i], coords[j]), dist))
    return strains

# === Draw Tension Map ===
def draw_tension_map(image, strain_data):
    tension_map = np.zeros_like(image)
    for ((pt1, pt2), val) in strain_data:
        intensity = int(min(max(val, 0), 50)) * 5  # Scale to 0–255
        cv2.line(tension_map, pt1, pt2, (0, 0, intensity), 2)
    heatmap = cv2.applyColorMap(tension_map[:, :, 2], cv2.COLORMAP_JET)
    blended = cv2.addWeighted(image, 0.6, heatmap, 0.4, 0)
    return blended

# === Process Each Video Folder ===
def process_video_folder(video_path, output_subfolder):
    os.makedirs(output_subfolder, exist_ok=True)
    for img_file in tqdm(sorted(os.listdir(video_path)), desc=f"Processing {os.path.basename(video_path)}"):
        if not img_file.endswith((".jpg", ".png")):
            continue
        img_path = os.path.join(video_path, img_file)
        image = cv2.imread(img_path)
        if image is None:
            continue
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb)
        if results.multi_face_landmarks:
            lm = results.multi_face_landmarks[0].landmark
            strain_data = compute_strain(lm, image.shape)
            tension_map = draw_tension_map(image, strain_data)
            save_path = os.path.join(output_subfolder, img_file)
            cv2.imwrite(save_path, tension_map)

# === Wrapper for Real and Fake Processing ===
def process_dataset(input_path, subfolder_name):
    out_dir = os.path.join(OUTPUT_FOLDER, subfolder_name)
    for video_id in os.listdir(input_path):
        video_folder = os.path.join(input_path, video_id)
        if os.path.isdir(video_folder):
            output_subfolder = os.path.join(out_dir, video_id)
            process_video_folder(video_folder, output_subfolder)

# === Run on Both Datasets ===
process_dataset(REAL_INPUT, "real")
process_dataset(FAKE_INPUT, "fake")

print("✅ Muscle tension maps saved to:", OUTPUT_FOLDER)


Processing 00032: 100%|██████████| 463/463 [00:09<00:00, 46.87it/s]
Processing 00033: 0it [00:00, ?it/s]
Processing 00133: 100%|██████████| 442/442 [00:14<00:00, 31.12it/s]
Processing 00134: 0it [00:00, ?it/s]
Processing 00163: 100%|██████████| 455/455 [00:14<00:00, 30.33it/s]
Processing 00164: 0it [00:00, ?it/s]
Processing id8_0000: 100%|██████████| 471/471 [00:11<00:00, 41.81it/s]
Processing id8_0001: 0it [00:00, ?it/s]
Processing id8_0007: 100%|██████████| 10/10 [00:00<00:00, 60.80it/s]
Processing id8_0008: 0it [00:00, ?it/s]
Processing id8_id1_0003: 100%|██████████| 388/388 [00:07<00:00, 50.02it/s]
Processing id8_id1_0007: 0it [00:00, ?it/s]
Processing id8_id5_0003: 100%|██████████| 384/384 [00:07<00:00, 50.91it/s]
Processing id8_id5_0007: 0it [00:00, ?it/s]
Processing id8_id5_0008: 0it [00:00, ?it/s]
Processing id9_id6_0009: 100%|██████████| 313/313 [00:09<00:00, 33.27it/s]

✅ Muscle tension maps saved to: D:/DETECTION 1.0/Celeb-DF/muscle_tension_maps


In [3]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

# === CONFIG ===
TENSION_MAPS_FOLDER = r"D:\DETECTION 1.0\Celeb-DF\muscle_tension_maps"
OUTPUT_CSV = "muscle_tension_features.csv"

# === Extract Histogram from Grayscale Image ===
def extract_histogram_features(gray_img, bins=32):
    hist = cv2.calcHist([gray_img], [0], None, [bins], [0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

# === Main Processing ===
all_features = []

for label_name in ['real', 'fake']:
    label_path = os.path.join(TENSION_MAPS_FOLDER, label_name)
    label_value = 0 if label_name == 'real' else 1

    if not os.path.isdir(label_path):
        print(f"⚠️ Folder not found: {label_path}")
        continue

    for video_id in tqdm(os.listdir(label_path), desc=f"📦 Processing {label_name} videos"):
        video_folder = os.path.join(label_path, video_id)
        if not os.path.isdir(video_folder):
            continue

        frame_files = sorted([f for f in os.listdir(video_folder) if f.lower().endswith(('.jpg', '.png'))])
        if not frame_files:
            continue

        video_hists = []
        for frame_file in frame_files:
            img_path = os.path.join(video_folder, frame_file)
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if image is None:
                continue
            hist = extract_histogram_features(image)
            video_hists.append(hist)

        if video_hists:
            avg_hist = np.mean(video_hists, axis=0)
            all_features.append({
                'video_id': video_id,
                'label': label_value,
                **{f'hist_bin_{i}': avg_hist[i] for i in range(len(avg_hist))}
            })

# === Save CSV ===
df = pd.DataFrame(all_features)
df.to_csv(OUTPUT_CSV, index=False)
print(f"\n✅ Saved muscle tension features to: {OUTPUT_CSV}")


📦 Processing fake videos: 100%|██████████| 408/408 [20:15<00:00,  2.98s/it]


✅ Saved muscle tension features to: muscle_tension_features.csv


In [15]:
import pandas as pd

# === Paths to CSV files ===
csv_files = {
    "Label": r"D:\DETECTION 1.0\Celeb-DF\Label Real and Fake.csv",
    "Motion": r"D:\DETECTION 1.0\Celeb-DF\combined_motion.csv",
    "NMCS": r"D:\DETECTION 1.0\Celeb-DF\nmcs_scores.csv",
    "Muscle": r"D:\DETECTION 1.0\Celeb-DF\muscle_tension_features.csv"
}

# === Load and normalize ===
def load_and_normalize(path, name):
    try:
        df = pd.read_csv(path)

        # Rename common column to 'video_id'
        for col in ['video', 'video_id', 'filename']:
            if col in df.columns:
                df = df.rename(columns={col: 'video_id'})
                break

        # Keep original ID format (to preserve things like id11_0005)
        df['video_id'] = df['video_id'].astype(str)

        # Label checks
        if 'label' not in df.columns:
            print(f"⚠️ {name} missing 'label' column.")
        else:
            labels = df['label'].dropna().unique()
            print(f"✅ {name} → Labels: {labels}")

        print(f"📦 {name} | Rows: {len(df)} | Unique video_ids: {df['video_id'].nunique()}")
        return df

    except Exception as e:
        print(f"❌ Failed to load {name}: {e}")
        return pd.DataFrame()

# === Load All CSVs ===
dfs = {}
for name, path in csv_files.items():
    dfs[name] = load_and_normalize(path, name)

# === Optional: Check common video_ids across all datasets ===
common_ids = set(dfs["Label"]['video_id'])
for name, df in dfs.items():
    if 'video_id' in df.columns:
        common_ids &= set(df['video_id'])

print(f"\n✅ Common video_ids across all datasets: {len(common_ids)}")

✅ Label → Labels: [1 0]
📦 Label | Rows: 347604 | Unique video_ids: 347604
✅ Motion → Labels: [0 1]
📦 Motion | Rows: 346788 | Unique video_ids: 346788
✅ NMCS → Labels: [0 1]
📦 NMCS | Rows: 797 | Unique video_ids: 797
✅ Muscle → Labels: [0 1]
📦 Muscle | Rows: 798 | Unique video_ids: 798

✅ Common video_ids across all datasets: 0


In [14]:
import pandas as pd

# Load your CSV file (replace with actual file if different)
df = pd.read_csv(r"D:\DETECTION 1.0\Celeb-DF\nmcs_scores.csv")

# Ensure 'video_id' is treated as string
df['video_id'] = df['video_id'].astype(str)

# Count occurrences
video_counts = df['video_id'].value_counts()

# Find repeated ones
repeated_videos = video_counts[video_counts > 1]

# Show the repeated video IDs
print("🔁 Repeated video_ids (treated distinctly):")
print(repeated_videos)
print(f"\n🧮 Total repeated videos: {len(repeated_videos)}")


🔁 Repeated video_ids (treated distinctly):
Series([], Name: count, dtype: int64)

🧮 Total repeated videos: 0


In [16]:
import pandas as pd

# === Paths to CSV files ===
csv_files = {
    "Label": r"D:\DETECTION 1.0\Celeb-DF\Label Real and Fake.csv",
    "Motion": r"D:\DETECTION 1.0\Celeb-DF\combined_motion.csv",
    "NMCS": r"D:\DETECTION 1.0\Celeb-DF\nmcs_scores.csv",
    "Muscle": r"D:\DETECTION 1.0\Celeb-DF\muscle_tension_features.csv"
}

# === Main Counting Function ===
def count_real_fake_videos(path, name):
    try:
        df = pd.read_csv(path)

        # Rename to standard column name
        for col in ['video', 'filename']:
            if col in df.columns:
                df = df.rename(columns={col: 'video_id'})
                break

        df['video_id'] = df['video_id'].astype(str)

        # Attach label if missing
        if 'label' not in df.columns:
            label_df = pd.read_csv(csv_files["Label"])[['video_id', 'label']]
            label_df['video_id'] = label_df['video_id'].astype(str)
            df = df.merge(label_df, on='video_id', how='left')

        # Drop duplicates for correct counting
        unique_df = df[['video_id', 'label']].drop_duplicates()
        label_counts = unique_df['label'].value_counts().sort_index()

        print(f"\n📦 {name}:")
        print(f"🟢 Real (0): {label_counts.get(0, 0)}")
        print(f"🔴 Fake (1): {label_counts.get(1, 0)}")
        print(f"🎞️ Total Unique Videos: {unique_df['video_id'].nunique()}")
        print("-" * 45)

    except Exception as e:
        print(f"❌ Error in {name}: {e}")

# === Run the Checks ===
for name, path in csv_files.items():
    count_real_fake_videos(path, name)



📦 Label:
🟢 Real (0): 175289
🔴 Fake (1): 172315
🎞️ Total Unique Videos: 347604
---------------------------------------------

📦 Motion:
🟢 Real (0): 174881
🔴 Fake (1): 171907
🎞️ Total Unique Videos: 346788
---------------------------------------------

📦 NMCS:
🟢 Real (0): 401
🔴 Fake (1): 396
🎞️ Total Unique Videos: 797
---------------------------------------------

📦 Muscle:
🟢 Real (0): 402
🔴 Fake (1): 396
🎞️ Total Unique Videos: 798
---------------------------------------------


In [8]:
import os

REAL_ALIGNED = r"D:\DETECTION 1.0\Celeb-DF\aligned_faces_real"
FAKE_ALIGNED = r"D:\DETECTION 1.0\Celeb-DF\aligned_faces_fake"

real_videos = [f for f in os.listdir(REAL_ALIGNED) if os.path.isdir(os.path.join(REAL_ALIGNED, f))]
fake_videos = [f for f in os.listdir(FAKE_ALIGNED) if os.path.isdir(os.path.join(FAKE_ALIGNED, f))]

print("✅ Total real aligned videos:", len(real_videos))
print("✅ Total fake aligned videos:", len(fake_videos))


✅ Total real aligned videos: 408
✅ Total fake aligned videos: 408


In [46]:
import pandas as pd

# === Paths to input CSVs ===
csv_files = {
    "Label": r"D:\DETECTION 1.0\Celeb-DF\Label Real and Fake.csv",
    "Motion": r"D:\DETECTION 1.0\Celeb-DF\combined_motion.csv",
    "NMCS": r"D:\DETECTION 1.0\Celeb-DF\nmcs_scores.csv",
    "Muscle": r"D:\DETECTION 1.0\Celeb-DF\muscle_tension_features.csv"
}

# === Load and Normalize ===
def load_csv(path, name):
    df = pd.read_csv(path)

    # Standardize column name
    for col in ['video', 'filename']:
        if col in df.columns:
            df = df.rename(columns={col: 'video_id'})
            break

    df['video_id'] = df['video_id'].astype(str)

    # If video_id is just digits like '1', '2', pad them to 5 digits
    if df['video_id'].str.match(r'^\d+$').all():
        df['video_id'] = df['video_id'].str.zfill(5)

    if 'label' not in df.columns and name != 'Label':
        label_df = pd.read_csv(csv_files["Label"])[['video_id', 'label']]
        label_df['video_id'] = label_df['video_id'].astype(str)
        if label_df['video_id'].str.match(r'^\d+$').all():
            label_df['video_id'] = label_df['video_id'].str.zfill(5)
        df = df.merge(label_df, on='video_id', how='left')

    return df


# === Load all datasets ===
label_df  = load_csv(csv_files["Label"], "Label")[['video_id', 'label']]
motion_df = load_csv(csv_files["Motion"], "Motion")
nmcs_df   = load_csv(csv_files["NMCS"], "NMCS")
muscle_df = load_csv(csv_files["Muscle"], "Muscle")

# === Group & Aggregate (in case multiple rows per video) ===
motion_agg = motion_df.groupby(['video_id', 'label'], as_index=False).mean(numeric_only=True)
nmcs_agg   = nmcs_df.groupby(['video_id', 'label'], as_index=False).mean(numeric_only=True)
muscle_agg = muscle_df.groupby(['video_id', 'label'], as_index=False).mean(numeric_only=True)

# === Merge all ===
merged_df = label_df.copy()
merged_df = merged_df.merge(motion_agg, on=['video_id', 'label'], how='inner')
merged_df = merged_df.merge(nmcs_agg,   on=['video_id', 'label'], how='inner')
merged_df = merged_df.merge(muscle_agg, on=['video_id', 'label'], how='inner')

# === Save output ===
output_path = r"D:\DETECTION 1.0\Celeb-DF\fused_features.csv"
merged_df.to_csv(output_path, index=False)

# === Info ===
print("\n✅ Fusion complete. Saved to:", output_path)
print(f"📦 Final shape: {merged_df.shape}")
print(f"🟢 Real: {(merged_df['label'] == 0).sum()} | 🔴 Fake: {(merged_df['label'] == 1).sum()}")



✅ Fusion complete. Saved to: D:\DETECTION 1.0\Celeb-DF\fused_features.csv
📦 Final shape: (0, 38)
🟢 Real: 0 | 🔴 Fake: 0


In [47]:
# Print first 5 video_ids from each CSV
print("\n🎯 Sample Video IDs:")
print("📌 Label  →", label_df['video_id'].unique()[:5])
print("📌 Motion →", motion_df['video_id'].unique()[:5])
print("📌 NMCS   →", nmcs_df['video_id'].unique()[:5])
print("📌 Muscle →", muscle_df['video_id'].unique()[:5])

# Check intersection size
sets = {
    "Label": set(label_df['video_id']),
    "Motion": set(motion_agg['video_id']),
    "NMCS": set(nmcs_agg['video_id']),
    "Muscle": set(muscle_agg['video_id']),
}
print("\n✅ Video ID Overlap Stats:")
print("🧩 Label ∩ Motion:", len(sets["Label"] & sets["Motion"]))
print("🧩 Label ∩ NMCS:", len(sets["Label"] & sets["NMCS"]))
print("🧩 Label ∩ Muscle:", len(sets["Label"] & sets["Muscle"]))
print("🧩 All 4 Intersecting:", len(set.intersection(*sets.values())))



🎯 Sample Video IDs:
📌 Label  → ['00000' '00001' '00002' '00003' '00004']
📌 Motion → ['00000' '00001' '00002' '00003' '00004']
📌 NMCS   → ['00000' '00001' '00002' '00003' '00004']
📌 Muscle → ['00000' '00001' '00002' '00003' '00004']

✅ Video ID Overlap Stats:
🧩 Label ∩ Motion: 346788
🧩 Label ∩ NMCS: 246
🧩 Label ∩ Muscle: 247
🧩 All 4 Intersecting: 246


In [48]:
# Show sample video_ids from each dataframe
print("\n🎯 Sample Video IDs:")
print("📌 Label  →", label_df['video_id'].dropna().astype(str).unique()[:5])
print("📌 Motion →", motion_df['video_id'].dropna().astype(str).unique()[:5])
print("📌 NMCS   →", nmcs_df['video_id'].dropna().astype(str).unique()[:5])
print("📌 Muscle →", muscle_df['video_id'].dropna().astype(str).unique()[:5])


🎯 Sample Video IDs:
📌 Label  → ['00000' '00001' '00002' '00003' '00004']
📌 Motion → ['00000' '00001' '00002' '00003' '00004']
📌 NMCS   → ['00000' '00001' '00002' '00003' '00004']
📌 Muscle → ['00000' '00001' '00002' '00003' '00004']


In [ ]:
import pandas as pd

# === Paths to input CSVs ===
csv_files = {
    "Label":  r"D:\DETECTION 1.0\Celeb-DF\Label Real and Fake.csv",
    "Motion": r"D:\DETECTION 1.0\Celeb-DF\combined_motion.csv",
    "NMCS":   r"D:\DETECTION 1.0\Celeb-DF\nmcs_scores.csv",
    "Muscle": r"D:\DETECTION 1.0\Celeb-DF\muscle_tension_features.csv"
}

# === Load and Normalize CSV ===
def load_csv(path, name):
    df = pd.read_csv(path)

    # Standardize video_id column
    for col in ['video', 'filename', 'video_id']:
        if col in df.columns:
            df = df.rename(columns={col: 'video_id'})
            break

    df['video_id'] = df['video_id'].astype(str).str.zfill(5)

    # Merge label for non-label files
    if 'label' not in df.columns and name != 'Label':
        label_df = pd.read_csv(csv_files["Label"])[['video_id', 'label']]
        label_df['video_id'] = label_df['video_id'].astype(str).str.zfill(5)
        df = df.merge(label_df, on='video_id', how='left')

    print(f"✅ Loaded {name}: shape = {df.shape}")
    return df


# === Load datasets ===
label_df  = load_csv(csv_files["Label"], "Label")[['video_id', 'label']]
motion_df = load_csv(csv_files["Motion"], "Motion")
nmcs_df   = load_csv(csv_files["NMCS"], "NMCS")
muscle_df = load_csv(csv_files["Muscle"], "Muscle")

# === Drop 'label' columns from features if present ===
for df in [motion_df, nmcs_df, muscle_df]:
    if 'label' in df.columns:
        df.drop(columns=['label'], inplace=True)

# === Group & Aggregate to video level ===
motion_agg = motion_df.groupby('video_id', as_index=False).mean(numeric_only=True)
nmcs_agg   = nmcs_df.groupby('video_id', as_index=False).mean(numeric_only=True)
muscle_agg = muscle_df.groupby('video_id', as_index=False).mean(numeric_only=True)

# === Merge all on video_id only ===
merged_df = label_df.copy()
merged_df = merged_df.merge(motion_agg, on='video_id', how='inner')
merged_df = merged_df.merge(nmcs_agg,   on='video_id', how='inner')
merged_df = merged_df.merge(muscle_agg, on='video_id', how='inner')

# === Save final fused features ===
output_path = r"D:\DETECTION 1.0\Celeb-DF\fused_features.csv"
merged_df.to_csv(output_path, index=False)

# === Print summary ===
print("\n✅ Fusion complete. Saved to:", output_path)
print(f"📦 Final shape: {merged_df.shape}")
print(f"🟢 Real: {(merged_df['label'] == 0).sum()} | 🔴 Fake: {(merged_df['label'] == 1).sum()}")

# === Optional: Show intersections ===
sets = {
    "Label":  set(label_df['video_id']),
    "Motion": set(motion_agg['video_id']),
    "NMCS":   set(nmcs_agg['video_id']),
    "Muscle": set(muscle_agg['video_id']),
}
print("\n✅ Video ID Overlap Stats:")
print("🧩 Label ∩ Motion:", len(sets["Label"] & sets["Motion"]))
print("🧩 Label ∩ NMCS:", len(sets["Label"] & sets["NMCS"]))
print("🧩 Label ∩ Muscle:", len(sets["Label"] & sets["Muscle"]))
print("🧩 All 4 Intersecting:", len(set.intersection(*sets.values())))


✅ NMCS features saved: D:\DETECTION 1.0\Celeb-DF\nmcs_scores_real.csv


In [56]:
import os
import pandas as pd

# === Input & Output Paths ===
real_frames_path = r"D:\DETECTION 1.0\Celeb-DF\Real Frames"
output_csv = r"D:\DETECTION 1.0\Celeb-DF\nmcs_scores_real.csv"

# === Dummy NMCS Computation (replace with actual logic) ===
def compute_nmcs_for_video(video_id, frames_folder):
    # Replace with actual NMCS computation
    return {
        'video_id': video_id,
        'nmcs_score': 0.85,  # placeholder
        'coord_variance': 0.12  # placeholder
    }

# === Process Real Videos ===
data = []
for video_id in sorted(os.listdir(real_frames_path)):
    video_folder = os.path.join(real_frames_path, video_id)
    if os.path.isdir(video_folder):
        features = compute_nmcs_for_video(video_id, video_folder)
        data.append(features)

# === Save Results ===
nmcs_df = pd.DataFrame(data)
nmcs_df['video_id'] = nmcs_df['video_id'].astype(str).str.zfill(5)
nmcs_df.to_csv(output_csv, index=False)
print("✅ NMCS features saved:", output_csv)


✅ NMCS features saved: D:\DETECTION 1.0\Celeb-DF\nmcs_scores_real.csv


In [57]:
import os
import pandas as pd

# === Input & Output Paths ===
real_frames_path = r"D:\DETECTION 1.0\Celeb-DF\Real Frames"
output_csv = r"D:\DETECTION 1.0\Celeb-DF\muscle_tension_features_real.csv"

# === Dummy Muscle Tension Computation (replace with actual logic) ===
def compute_muscle_features(video_id, frames_folder):
    return {
        'video_id': video_id,
        'jaw_strain': 0.21,
        'cheek_strain': 0.18,
        'forehead_strain': 0.12,
        # add all 30+ fields...
    }

# === Process Real Videos ===
data = []
for video_id in sorted(os.listdir(real_frames_path)):
    video_folder = os.path.join(real_frames_path, video_id)
    if os.path.isdir(video_folder):
        features = compute_muscle_features(video_id, video_folder)
        data.append(features)

# === Save Results ===
muscle_df = pd.DataFrame(data)
muscle_df['video_id'] = muscle_df['video_id'].astype(str).str.zfill(5)
muscle_df.to_csv(output_csv, index=False)
print("✅ Muscle Tension features saved:", output_csv)


✅ Muscle Tension features saved: D:\DETECTION 1.0\Celeb-DF\muscle_tension_features_real.csv


In [60]:
import pandas as pd

# === Paths ===
nmcs_real_path   = r"D:\DETECTION 1.0\Celeb-DF\nmcs_scores_real.csv"
nmcs_full_path   = r"D:\DETECTION 1.0\Celeb-DF\nmcs_scores.csv"
muscle_real_path = r"D:\DETECTION 1.0\Celeb-DF\muscle_tension_features_real.csv"
muscle_full_path = r"D:\DETECTION 1.0\Celeb-DF\muscle_tension_features.csv"

# === Load Real and Fake Features ===
nmcs_real   = pd.read_csv(nmcs_real_path)
nmcs_fake   = pd.read_csv(nmcs_full_path)
muscle_real = pd.read_csv(muscle_real_path)
muscle_fake = pd.read_csv(muscle_full_path)

# === Combine and Deduplicate if necessary ===
nmcs_combined   = pd.concat([nmcs_fake, nmcs_real], ignore_index=True).drop_duplicates(subset='video_id')
muscle_combined = pd.concat([muscle_fake, muscle_real], ignore_index=True).drop_duplicates(subset='video_id')

# === Save back to full feature CSVs ===
nmcs_combined.to_csv(nmcs_full_path, index=False)
muscle_combined.to_csv(muscle_full_path, index=False)

print("✅ NMCS and Muscle files updated with real video features.")


✅ NMCS and Muscle files updated with real video features.


In [61]:
import pandas as pd

# === File paths ===
nmcs_real_path   = r"D:\DETECTION 1.0\Celeb-DF\nmcs_scores_real.csv"
nmcs_full_path   = r"D:\DETECTION 1.0\Celeb-DF\nmcs_scores.csv"

muscle_real_path = r"D:\DETECTION 1.0\Celeb-DF\muscle_tension_features_real.csv"
muscle_full_path = r"D:\DETECTION 1.0\Celeb-DF\muscle_tension_features.csv"

# === Load CSVs ===
nmcs_real   = pd.read_csv(nmcs_real_path)
nmcs_full   = pd.read_csv(nmcs_full_path)
muscle_real = pd.read_csv(muscle_real_path)
muscle_full = pd.read_csv(muscle_full_path)

# === Normalize video_id ===
for df in [nmcs_real, nmcs_full, muscle_real, muscle_full]:
    df['video_id'] = df['video_id'].astype(str).str.zfill(5)

# === Concatenate & drop duplicates ===
nmcs_combined   = pd.concat([nmcs_full, nmcs_real], ignore_index=True).drop_duplicates(subset='video_id')
muscle_combined = pd.concat([muscle_full, muscle_real], ignore_index=True).drop_duplicates(subset='video_id')

# === Save back to original files ===
nmcs_combined.to_csv(nmcs_full_path, index=False)
muscle_combined.to_csv(muscle_full_path, index=False)

print("✅ Successfully appended real NMCS and Muscle features using pandas.")


✅ Successfully appended real NMCS and Muscle features using pandas.


In [62]:
import pandas as pd

# === Input Paths ===
csv_files = {
    "Label":  r"D:\DETECTION 1.0\Celeb-DF\Label Real and Fake.csv",
    "Motion": r"D:\DETECTION 1.0\Celeb-DF\combined_motion.csv",
    "NMCS":   r"D:\DETECTION 1.0\Celeb-DF\nmcs_scores.csv",
    "Muscle": r"D:\DETECTION 1.0\Celeb-DF\muscle_tension_features.csv"
}

# === Load CSV and Normalize ===
def load_csv(path, name):
    df = pd.read_csv(path)
    for col in ['video', 'filename', 'video_id']:
        if col in df.columns:
            df = df.rename(columns={col: 'video_id'})
            break
    df['video_id'] = df['video_id'].astype(str).str.zfill(5)

    if 'label' not in df.columns and name != 'Label':
        label_df = pd.read_csv(csv_files["Label"])[['video_id', 'label']]
        label_df['video_id'] = label_df['video_id'].astype(str).str.zfill(5)
        df = df.merge(label_df, on='video_id', how='left')

    print(f"✅ Loaded {name}: shape = {df.shape}")
    return df

# === Load and clean datasets ===
label_df  = load_csv(csv_files["Label"], "Label")[['video_id', 'label']]
motion_df = load_csv(csv_files["Motion"], "Motion")
nmcs_df   = load_csv(csv_files["NMCS"], "NMCS")
muscle_df = load_csv(csv_files["Muscle"], "Muscle")

# === Drop any duplicate 'label' columns from features
for df in [motion_df, nmcs_df, muscle_df]:
    if 'label' in df.columns:
        df.drop(columns=['label'], inplace=True)

# === Group by video_id (if needed)
motion_agg = motion_df.groupby('video_id', as_index=False).mean(numeric_only=True)
nmcs_agg   = nmcs_df.groupby('video_id', as_index=False).mean(numeric_only=True)
muscle_agg = muscle_df.groupby('video_id', as_index=False).mean(numeric_only=True)

# === Merge everything
merged_df = label_df.copy()
merged_df = merged_df.merge(motion_agg, on='video_id', how='inner')
merged_df = merged_df.merge(nmcs_agg,   on='video_id', how='inner')
merged_df = merged_df.merge(muscle_agg, on='video_id', how='inner')

# === Save final output
output_path = r"D:\DETECTION 1.0\Celeb-DF\fused_features.csv"
merged_df.to_csv(output_path, index=False)

# === Summary
print("\n✅ Fusion complete. Saved to:", output_path)
print(f"📦 Final shape: {merged_df.shape}")
print(f"🟢 Real: {(merged_df['label'] == 0).sum()} | 🔴 Fake: {(merged_df['label'] == 1).sum()}")

# === Optional: Intersection Stats
sets = {
    "Label":  set(label_df['video_id']),
    "Motion": set(motion_agg['video_id']),
    "NMCS":   set(nmcs_agg['video_id']),
    "Muscle": set(muscle_agg['video_id']),
}
print("\n✅ Video ID Overlap Stats:")
print("🧩 Label ∩ Motion:", len(sets["Label"] & sets["Motion"]))
print("🧩 Label ∩ NMCS:", len(sets["Label"] & sets["NMCS"]))
print("🧩 Label ∩ Muscle:", len(sets["Label"] & sets["Muscle"]))
print("🧩 All 4 Intersecting:", len(set.intersection(*sets.values())))


✅ Loaded Label: shape = (347604, 3)
✅ Loaded Motion: shape = (346788, 6)
✅ Loaded NMCS: shape = (804, 5)
✅ Loaded Muscle: shape = (804, 37)

✅ Fusion complete. Saved to: D:\DETECTION 1.0\Celeb-DF\fused_features.csv
📦 Final shape: (250, 43)
🟢 Real: 0 | 🔴 Fake: 250

✅ Video ID Overlap Stats:
🧩 Label ∩ Motion: 346788
🧩 Label ∩ NMCS: 250
🧩 Label ∩ Muscle: 250
🧩 All 4 Intersecting: 250


In [70]:
import pandas as pd

# === Paths to input CSVs ===
csv_files = {
    "Label":  r'D:\DETECTION 1.0\Celeb-DF\aligned_frame_labels.csv',
    "Motion": r"D:\DETECTION 1.0\Celeb-DF\combined_motion.csv",
    "NMCS":   r"D:\DETECTION 1.0\Celeb-DF\nmcs_scores.csv",
    "Muscle": r"D:\DETECTION 1.0\Celeb-DF\muscle_tension_features.csv"
}

# === Load and Normalize ===
def load_csv(path, name):
    df = pd.read_csv(path)

    for col in ['video', 'filename', 'video_id']:
        if col in df.columns:
            df = df.rename(columns={col: 'video_id'})
            break

    df['video_id'] = df['video_id'].astype(str)

    if 'label' not in df.columns and name != 'Label':
        label_df = pd.read_csv(csv_files["Label"])[['video_id', 'label']]
        label_df['video_id'] = label_df['video_id'].astype(str)
        df = df.merge(label_df, on='video_id', how='left')

    print(f"✅ Loaded {name}: shape = {df.shape}")
    return df

# === Load all datasets ===
label_df  = load_csv(csv_files["Label"],  "Label")[['video_id', 'label']]
motion_df = load_csv(csv_files["Motion"], "Motion")
nmcs_df   = load_csv(csv_files["NMCS"],   "NMCS")
muscle_df = load_csv(csv_files["Muscle"], "Muscle")

# === Drop label column from features if present
for df in [motion_df, nmcs_df, muscle_df]:
    if 'label' in df.columns:
        df.drop(columns=['label'], inplace=True)

# === Group (in case multiple rows per video)
motion_agg = motion_df.groupby('video_id', as_index=False).mean(numeric_only=True)
nmcs_agg   = nmcs_df.groupby('video_id', as_index=False).mean(numeric_only=True)
muscle_agg = muscle_df.groupby('video_id', as_index=False).mean(numeric_only=True)

# === Merge
merged_df = label_df.copy()
merged_df = merged_df.merge(motion_agg, on='video_id', how='inner')
merged_df = merged_df.merge(nmcs_agg,   on='video_id', how='inner')
merged_df = merged_df.merge(muscle_agg, on='video_id', how='inner')

# === Save
output_path = r"D:\DETECTION 1.0\Celeb-DF\fused_features.csv"
merged_df.to_csv(output_path, index=False)

# === Summary
print("\n✅ Fusion complete. Saved to:", output_path)
print(f"📦 Final shape: {merged_df.shape}")
print(f"🟢 Real: {(merged_df['label'] == 0).sum()} | 🔴 Fake: {(merged_df['label'] == 1).sum()}")

# === Overlap Stats (optional)
sets = {
    "Label":  set(label_df['video_id']),
    "Motion": set(motion_agg['video_id']),
    "NMCS":   set(nmcs_agg['video_id']),
    "Muscle": set(muscle_agg['video_id']),
}
print("\n✅ Video ID Overlap Stats:")
print("🧩 Label ∩ Motion:", len(sets["Label"] & sets["Motion"]))
print("🧩 Label ∩ NMCS:", len(sets["Label"] & sets["NMCS"]))
print("🧩 Label ∩ Muscle:", len(sets["Label"] & sets["Muscle"]))
print("🧩 All 4 Intersecting:", len(set.intersection(*sets.values())))


✅ Loaded Label: shape = (310866, 3)
✅ Loaded Motion: shape = (346788, 6)
✅ Loaded NMCS: shape = (804, 5)
✅ Loaded Muscle: shape = (804, 37)

✅ Fusion complete. Saved to: D:\DETECTION 1.0\Celeb-DF\fused_features.csv
📦 Final shape: (0, 43)
🟢 Real: 0 | 🔴 Fake: 0

✅ Video ID Overlap Stats:
🧩 Label ∩ Motion: 0
🧩 Label ∩ NMCS: 799
🧩 Label ∩ Muscle: 799
🧩 All 4 Intersecting: 0


In [69]:
import os
import csv

# Define input and output paths
real_frame_path = r"D:\DETECTION 1.0\Celeb-DF\aligned_faces_real"
fake_frame_path = r"D:\DETECTION 1.0\Celeb-DF\aligned_faces_fake"
output_csv_path = r'D:\DETECTION 1.0\Celeb-DF\aligned_frame_labels.csv'

# Allowed image extensions
valid_extensions = ('.jpg', '.jpeg', '.png')

# Helper to get relative subfolder name (video_id)
def extract_video_id(full_path, base_path):
    rel_path = os.path.relpath(full_path, base_path)
    parts = rel_path.split(os.sep)
    if len(parts) >= 2:
        return parts[0].zfill(5)  # Ensure 5-digit ID
    return "00000"  # fallback (shouldn't happen)

# Function to collect (video_id, path, label)
def collect_data(base_path, label):
    data = []
    for root, _, files in os.walk(base_path):
        for file in files:
            if file.lower().endswith(valid_extensions):
                full_path = os.path.join(root, file)
                video_id = extract_video_id(full_path, base_path)
                data.append([video_id, full_path, label])
    return data

# Collect both real and fake
real_data = collect_data(real_frame_path, 0)
fake_data = collect_data(fake_frame_path, 1)

# Combine and sort
all_data = real_data + fake_data
all_data.sort()

# Write to CSV
with open(output_csv_path, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['video_id', 'filepath', 'label'])
    writer.writerows(all_data)

# Report
print(f"[✓] CSV saved: {output_csv_path}")
print(f"🟢 Real frames: {len(real_data)}")
print(f"🔴 Fake frames: {len(fake_data)}")
print(f"📦 Total frames: {len(all_data)}")


[✓] CSV saved: D:\DETECTION 1.0\Celeb-DF\aligned_frame_labels.csv
🟢 Real frames: 157205
🔴 Fake frames: 153661
📦 Total frames: 310866


In [71]:
import os
import glob
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

In [72]:
# ========== Step 1: Define utility functions ==========
def compute_dense_optical_flow(prev_img, next_img):
    prev_gray = cv2.cvtColor(prev_img, cv2.COLOR_BGR2GRAY)
    next_gray = cv2.cvtColor(next_img, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(
        prev_gray, next_gray, None,
        pyr_scale=0.5, levels=3, winsize=15,
        iterations=3, poly_n=5, poly_sigma=1.2, flags=0)
    return flow


In [73]:
def extract_zone_flow_features(flow, zone_masks):
    stats = {}
    for zone_name, mask in zone_masks.items():
        zone_flow = flow * mask[..., None]
        mag = np.linalg.norm(zone_flow, axis=2)
        zone_mag = mag[mask > 0]
        if zone_mag.size == 0:
            stats[zone_name] = {"mean": 0, "std": 0, "max": 0, "min": 0}
        else:
            stats[zone_name] = {
                "mean": np.mean(zone_mag),
                "std": np.std(zone_mag),
                "max": np.max(zone_mag),
                "min": np.min(zone_mag),
            }
    return stats

In [74]:
def save_optical_flow_heatmap(flow, save_path):
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv = np.zeros((flow.shape[0], flow.shape[1], 3), dtype=np.uint8)
    hsv[..., 0] = ang * 180 / np.pi / 2
    hsv[..., 1] = 255
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    cv2.imwrite(save_path, rgb)

In [75]:

def process_frame_pair(prev_img, next_img, zone_masks, save_dir, frame_id):
    flow = compute_dense_optical_flow(prev_img, next_img)
    stats = extract_zone_flow_features(flow, zone_masks)
    heatmap_path = os.path.join(save_dir, f"flow_heatmap_{frame_id}.png")
    save_optical_flow_heatmap(flow, heatmap_path)
    return stats

In [76]:
def count_total_frames(root_dir):
    total = 0
    for sub in os.listdir(root_dir):
        sub_dir = os.path.join(root_dir, sub)
        if os.path.isdir(sub_dir):
            frame_paths = glob.glob(os.path.join(sub_dir, '*.jpg')) + glob.glob(os.path.join(sub_dir, '*.png'))
            total += len(frame_paths)
    return total

In [77]:
# ========== Step 2: Process a folder of frames ==========
def process_video_folder(frame_dir, save_dir, label):
    os.makedirs(save_dir, exist_ok=True)
    frame_paths = sorted(glob.glob(os.path.join(frame_dir, '*.jpg')) +
                         glob.glob(os.path.join(frame_dir, '*.png')))

    frames = [cv2.imread(f) for f in frame_paths if cv2.imread(f) is not None]
    print(f"📂 {frame_dir} | Total frames: {len(frames)}")

    if len(frames) < 2:
        print("⚠️ Not enough frames to compute optical flow.")
        return pd.DataFrame()

    h, w = frames[0].shape[:2]
    zone_masks = {"full_face": np.ones((h, w), dtype=np.uint8)}

    motion_features = []
    for i in tqdm(range(len(frames) - 1), desc=f"Processing {os.path.basename(frame_dir)}"):
        stats = process_frame_pair(frames[i], frames[i+1], zone_masks, save_dir, i)
        flat_stats = {f"{zone}_{k}": v for zone, d in stats.items() for k, v in d.items()}
        flat_stats['pair_id'] = i
        flat_stats['label'] = label
        motion_features.append(flat_stats)

    return pd.DataFrame(motion_features)


In [78]:
# ========== Step 3: Provide root paths ==========
real_frame_path = r"D:\DETECTION 1.0\Celeb-DF\aligned_faces_real"
fake_frame_path = r"D:\DETECTION 1.0\Celeb-DF\aligned_faces_fake"

real_total = count_total_frames(real_frame_path)
fake_total = count_total_frames(fake_frame_path)
print(f"🟢 Total real frames: {real_total}")
print(f"🔴 Total fake frames: {fake_total}")

🟢 Total real frames: 157205
🔴 Total fake frames: 153661


In [79]:
# ========== Step 4: Process all subfolders ==========
real_dfs, fake_dfs = [], []
for sub in os.listdir(real_frame_path):
    path = os.path.join(real_frame_path, sub)
    if os.path.isdir(path):
        real_dfs.append(process_video_folder(path, f"heatmaps_real/{sub}", label=0))

for sub in os.listdir(fake_frame_path):
    path = os.path.join(fake_frame_path, sub)
    if os.path.isdir(path):
        fake_dfs.append(process_video_folder(path, f"heatmaps_fake/{sub}", label=1))


📂 D:\DETECTION 1.0\Celeb-DF\aligned_faces_real\00000 | Total frames: 450


Processing 00000: 100%|██████████| 449/449 [00:06<00:00, 68.41it/s]


📂 D:\DETECTION 1.0\Celeb-DF\aligned_faces_real\00001 | Total frames: 415


Processing 00001: 100%|██████████| 414/414 [00:06<00:00, 61.62it/s]


📂 D:\DETECTION 1.0\Celeb-DF\aligned_faces_real\00002 | Total frames: 410


...

Processing id9_id6_0007: 100%|██████████| 358/358 [00:14<00:00, 24.77it/s]


📂 D:\DETECTION 1.0\Celeb-DF\aligned_faces_fake\id9_id6_0008 | Total frames: 461


Processing id9_id6_0008: 100%|██████████| 460/460 [00:19<00:00, 23.60it/s]


📂 D:\DETECTION 1.0\Celeb-DF\aligned_faces_fake\id9_id6_0009 | Total frames: 313


Processing id9_id6_0009: 100%|██████████| 312/312 [00:13<00:00, 22.93it/s]


In [80]:
import glob

# Set paths to your heatmap folders
real_heatmap_dir = "heatmaps_real"
fake_heatmap_dir = "heatmaps_fake"

# Recursively count all PNG files
real_heatmaps = glob.glob(f"{real_heatmap_dir}/**/*.png", recursive=True)
fake_heatmaps = glob.glob(f"{fake_heatmap_dir}/**/*.png", recursive=True)

# Print results
print(f"🖼️ Total real heatmaps saved: {len(real_heatmaps)}")
print(f"🖼️ Total fake heatmaps saved: {len(fake_heatmaps)}")


🖼️ Total real heatmaps saved: 174881
🖼️ Total fake heatmaps saved: 171907


In [81]:
import cv2
import numpy as np
import os
import glob
import pandas as pd
from tqdm import tqdm

def compute_flow_stats_from_heatmap(image_path):
    """Compute mean and std of flow intensity from a heatmap."""
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None  # Skip unreadable image
    mean_val = np.mean(img)
    std_val = np.std(img)
    return {
        'heatmap_path': image_path,
        'full_face_mean': mean_val,
        'full_face_std': std_val
    }

def process_heatmap_folder(folder_path, label):
    """Process all .png heatmaps in a folder and extract motion stats."""
    heatmap_paths = sorted(glob.glob(os.path.join(folder_path, '**', '*.png'), recursive=True))
    data = []
    for i, path in enumerate(tqdm(heatmap_paths, desc=f"Processing {folder_path}")):
        stats = compute_flow_stats_from_heatmap(path)
        if stats:
            stats['pair_id'] = i
            stats['label'] = label
            data.append(stats)
    return pd.DataFrame(data)

# Set your paths
real_heatmap_dir = "heatmaps_real"
fake_heatmap_dir = "heatmaps_fake"

# Process both sets
real_df = process_heatmap_folder(real_heatmap_dir, label=0)
fake_df = process_heatmap_folder(fake_heatmap_dir, label=1)

# Save individually
real_df.to_csv("real_motion.csv", index=False)
fake_df.to_csv("fake_motion.csv", index=False)

# Combine both
combined_df = pd.concat([real_df, fake_df], ignore_index=True)
combined_df.to_csv("combined_motion.csv", index=False)

# Summary
print(f"✅ real_motion.csv: {len(real_df)} rows")
print(f"✅ fake_motion.csv: {len(fake_df)} rows")
print(f"✅ combined_motion.csv: {len(combined_df)} rows")


Processing heatmaps_fake: 100%|██████████| 171907/171907 [37:09<00:00, 77.10it/s] 


✅ real_motion.csv: 174881 rows
✅ fake_motion.csv: 171907 rows
✅ combined_motion.csv: 346788 rows


In [1]:
import pandas as pd

# === Paths to input CSVs ===
csv_files = {
    "Label":  r"D:\DETECTION 1.0\Celeb-DF\Label Real and Fake.csv",
    "Motion": r"D:\DETECTION 1.0\Celeb-DF\combined_motion.csv",
    "NMCS":   r"D:\DETECTION 1.0\Celeb-DF\nmcs_scores_real.csv",
    "Muscle": r"D:\DETECTION 1.0\Celeb-DF\muscle_tension_features_real.csv"
}

# === Load and Normalize ===
def load_csv(path, name):
    df = pd.read_csv(path)

    for col in ['video', 'filename', 'video_id']:
        if col in df.columns:
            df = df.rename(columns={col: 'video_id'})
            break

    df['video_id'] = df['video_id'].astype(str)

    if 'label' not in df.columns and name != 'Label':
        label_df = pd.read_csv(csv_files["Label"])[['video_id', 'label']]
        label_df['video_id'] = label_df['video_id'].astype(str)
        df = df.merge(label_df, on='video_id', how='left')

    print(f"✅ Loaded {name}: shape = {df.shape}")
    return df

# === Load all datasets ===
label_df  = load_csv(csv_files["Label"],  "Label")[['video_id', 'label']]
motion_df = load_csv(csv_files["Motion"], "Motion")
nmcs_df   = load_csv(csv_files["NMCS"],   "NMCS")
muscle_df = load_csv(csv_files["Muscle"], "Muscle")

# === Drop label column from features if present
for df in [motion_df, nmcs_df, muscle_df]:
    if 'label' in df.columns:
        df.drop(columns=['label'], inplace=True)

# === Group (in case multiple rows per video)
motion_agg = motion_df.groupby('video_id', as_index=False).mean(numeric_only=True)
nmcs_agg   = nmcs_df.groupby('video_id', as_index=False).mean(numeric_only=True)
muscle_agg = muscle_df.groupby('video_id', as_index=False).mean(numeric_only=True)

# === Merge
merged_df = label_df.copy()
merged_df = merged_df.merge(motion_agg, on='video_id', how='inner')
merged_df = merged_df.merge(nmcs_agg,   on='video_id', how='inner')
merged_df = merged_df.merge(muscle_agg, on='video_id', how='inner')

# === Save
output_path = r"D:\DETECTION 1.0\Celeb-DF\fused_features.csv"
merged_df.to_csv(output_path, index=False)

# === Summary
print("\n✅ Fusion complete. Saved to:", output_path)
print(f"📦 Final shape: {merged_df.shape}")
print(f"🟢 Real: {(merged_df['label'] == 0).sum()} | 🔴 Fake: {(merged_df['label'] == 1).sum()}")

# === Overlap Stats (optional)
sets = {
    "Label":  set(label_df['video_id']),
    "Motion": set(motion_agg['video_id']),
    "NMCS":   set(nmcs_agg['video_id']),
    "Muscle": set(muscle_agg['video_id']),
}
print("\n✅ Video ID Overlap Stats:")
print("🧩 Label ∩ Motion:", len(sets["Label"] & sets["Motion"]))
print("🧩 Label ∩ NMCS:", len(sets["Label"] & sets["NMCS"]))
print("🧩 Label ∩ Muscle:", len(sets["Label"] & sets["Muscle"]))
print("🧩 All 4 Intersecting:", len(set.intersection(*sets.values())))


✅ Loaded Label: shape = (347604, 3)


KeyError: 'video_id'

In [1]:
import cv2
import numpy as np
import os
import glob
import pandas as pd
from tqdm import tqdm

def compute_flow_stats_from_heatmap(image_path):
    """Compute mean and std of flow intensity from a heatmap."""
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None  # Skip unreadable image
    mean_val = np.mean(img)
    std_val = np.std(img)
    return {
        'heatmap_path': image_path,
        'full_face_mean': mean_val,
        'full_face_std': std_val,
        'video_id': os.path.basename(os.path.dirname(image_path))  # <-- Extract video ID
    }

def process_heatmap_folder(folder_path, label):
    """Process all .png heatmaps in a folder and extract motion stats."""
    heatmap_paths = sorted(glob.glob(os.path.join(folder_path, '**', '*.png'), recursive=True))
    data = []
    for i, path in enumerate(tqdm(heatmap_paths, desc=f"Processing {folder_path}")):
        stats = compute_flow_stats_from_heatmap(path)
        if stats:
            stats['pair_id'] = i
            stats['label'] = label
            data.append(stats)
    return pd.DataFrame(data)

# Set your paths
real_heatmap_dir = "heatmaps_real"
fake_heatmap_dir = "heatmaps_fake"

# Process both sets
real_df = process_heatmap_folder(real_heatmap_dir, label=0)
fake_df = process_heatmap_folder(fake_heatmap_dir, label=1)

# Save individually
real_df.to_csv("real_motion.csv", index=False)
fake_df.to_csv("fake_motion.csv", index=False)

# Combine both
combined_df = pd.concat([real_df, fake_df], ignore_index=True)
combined_df.to_csv("combined_motion.csv", index=False)

# Summary
print(f"✅ real_motion.csv: {len(real_df)} rows")
print(f"✅ fake_motion.csv: {len(fake_df)} rows")
print(f"✅ combined_motion.csv: {len(combined_df)} rows")


Processing heatmaps_fake: 100%|██████████| 171907/171907 [34:50<00:00, 82.23it/s] 


✅ real_motion.csv: 174881 rows
✅ fake_motion.csv: 171907 rows
✅ combined_motion.csv: 346788 rows


In [2]:
import pandas as pd

# === Paths to input CSVs ===
csv_files = {
    "Label":  r"D:\DETECTION 1.0\Celeb-DF\Label Real and Fake.csv",
    "Motion": r"D:\DETECTION 1.0\Celeb-DF\combined_motion.csv",
    "NMCS":   r"D:\DETECTION 1.0\Celeb-DF\nmcs_scores.csv",
    "Muscle": r"D:\DETECTION 1.0\Celeb-DF\muscle_tension_features.csv"
}

# === Load and Normalize ===
def load_csv(path, name):
    df = pd.read_csv(path)

    for col in ['video', 'filename', 'video_id']:
        if col in df.columns:
            df = df.rename(columns={col: 'video_id'})
            break

    df['video_id'] = df['video_id'].astype(str)

    if 'label' not in df.columns and name != 'Label':
        label_df = pd.read_csv(csv_files["Label"])[['video_id', 'label']]
        label_df['video_id'] = label_df['video_id'].astype(str)
        df = df.merge(label_df, on='video_id', how='left')

    print(f"✅ Loaded {name}: shape = {df.shape}")
    return df

# === Load all datasets ===
label_df  = load_csv(csv_files["Label"],  "Label")[['video_id', 'label']]
motion_df = load_csv(csv_files["Motion"], "Motion")
nmcs_df   = load_csv(csv_files["NMCS"],   "NMCS")
muscle_df = load_csv(csv_files["Muscle"], "Muscle")

# === Drop label column from features if present
for df in [motion_df, nmcs_df, muscle_df]:
    if 'label' in df.columns:
        df.drop(columns=['label'], inplace=True)

# === Group (in case multiple rows per video)
motion_agg = motion_df.groupby('video_id', as_index=False).mean(numeric_only=True)
nmcs_agg   = nmcs_df.groupby('video_id', as_index=False).mean(numeric_only=True)
muscle_agg = muscle_df.groupby('video_id', as_index=False).mean(numeric_only=True)

# === Merge
merged_df = label_df.copy()
merged_df = merged_df.merge(motion_agg, on='video_id', how='inner')
merged_df = merged_df.merge(nmcs_agg,   on='video_id', how='inner')
merged_df = merged_df.merge(muscle_agg, on='video_id', how='inner')

# === Save
output_path = r"D:\DETECTION 1.0\Celeb-DF\fused_features.csv"
merged_df.to_csv(output_path, index=False)

# === Summary
print("\n✅ Fusion complete. Saved to:", output_path)
print(f"📦 Final shape: {merged_df.shape}")
print(f"🟢 Real: {(merged_df['label'] == 0).sum()} | 🔴 Fake: {(merged_df['label'] == 1).sum()}")

# === Overlap Stats (optional)
sets = {
    "Label":  set(label_df['video_id']),
    "Motion": set(motion_agg['video_id']),
    "NMCS":   set(nmcs_agg['video_id']),
    "Muscle": set(muscle_agg['video_id']),
}
print("\n✅ Video ID Overlap Stats:")
print("🧩 Label ∩ Motion:", len(sets["Label"] & sets["Motion"]))
print("🧩 Label ∩ NMCS:", len(sets["Label"] & sets["NMCS"]))
print("🧩 Label ∩ Muscle:", len(sets["Label"] & sets["Muscle"]))
print("🧩 All 4 Intersecting:", len(set.intersection(*sets.values())))


✅ Loaded Label: shape = (347604, 3)
✅ Loaded Motion: shape = (346788, 6)
✅ Loaded NMCS: shape = (804, 5)
✅ Loaded Muscle: shape = (804, 37)

✅ Fusion complete. Saved to: D:\DETECTION 1.0\Celeb-DF\fused_features.csv
📦 Final shape: (0, 43)
🟢 Real: 0 | 🔴 Fake: 0

✅ Video ID Overlap Stats:
🧩 Label ∩ Motion: 0
🧩 Label ∩ NMCS: 0
🧩 Label ∩ Muscle: 0
🧩 All 4 Intersecting: 0


In [3]:
import pandas as pd

# === File Paths ===
label_path = r"D:\DETECTION 1.0\Celeb-DF\aligned_frame_labels.csv"
motion_path = r"D:\DETECTION 1.0\Celeb-DF\combined_motion.csv"
nmcs_path = r"D:\DETECTION 1.0\Celeb-DF\nmcs_scores.csv"
muscle_path = r"D:\DETECTION 1.0\Celeb-DF\muscle_tension_features.csv"
output_path = r"D:\DETECTION 1.0\Celeb-DF\fused_features_new.csv"

# === Load CSVs ===
label_df = pd.read_csv(label_path)
motion_df = pd.read_csv(motion_path)
nmcs_df = pd.read_csv(nmcs_path)
muscle_df = pd.read_csv(muscle_path)

print(f"✅ Loaded Label: shape = {label_df.shape}")
print(f"✅ Loaded Motion: shape = {motion_df.shape}")
print(f"✅ Loaded NMCS: shape = {nmcs_df.shape}")
print(f"✅ Loaded Muscle: shape = {muscle_df.shape}")

# === Normalize video_id (keep only base video id like 00000) ===
label_df['video_id'] = label_df['video_id'].astype(str).apply(lambda x: x.split('_')[0])
motion_df['video_id'] = motion_df['video_id'].astype(str).apply(lambda x: x.split('_')[0])
nmcs_df['video_id'] = nmcs_df['video_id'].astype(str)
muscle_df['video_id'] = muscle_df['video_id'].astype(str)

# === Group frame-based data to video-level by averaging ===
label_grouped = label_df.groupby('video_id').mean(numeric_only=True).reset_index()
motion_grouped = motion_df.groupby('video_id').mean(numeric_only=True).reset_index()

# === Check intersections ===
label_ids = set(label_grouped['video_id'])
motion_ids = set(motion_grouped['video_id'])
nmcs_ids = set(nmcs_df['video_id'])
muscle_ids = set(muscle_df['video_id'])

intersect_all = label_ids & motion_ids & nmcs_ids & muscle_ids
print(f"\n✅ Video ID Overlap Stats:")
print(f"🧩 Label ∩ Motion: {len(label_ids & motion_ids)}")
print(f"🧩 Label ∩ NMCS: {len(label_ids & nmcs_ids)}")
print(f"🧩 Label ∩ Muscle: {len(label_ids & muscle_ids)}")
print(f"🧩 All 4 Intersecting: {len(intersect_all)}")

# === Filter all to common video_ids only ===
label_grouped = label_grouped[label_grouped['video_id'].isin(intersect_all)]
motion_grouped = motion_grouped[motion_grouped['video_id'].isin(intersect_all)]
nmcs_df = nmcs_df[nmcs_df['video_id'].isin(intersect_all)]
muscle_df = muscle_df[muscle_df['video_id'].isin(intersect_all)]

# === Safety check: ensure uniqueness ===
assert label_grouped['video_id'].is_unique
assert motion_grouped['video_id'].is_unique
assert nmcs_df['video_id'].is_unique
assert muscle_df['video_id'].is_unique

# === Merge all ===
fused = label_grouped.merge(motion_grouped, on='video_id') \
                     .merge(nmcs_df, on='video_id') \
                     .merge(muscle_df, on='video_id')

# === Save ===
fused.to_csv(output_path, index=False)
print(f"\n✅ Fusion complete. Saved to: {output_path}")
print(f"📦 Final shape: {fused.shape}")
print(f"🟢 Real: {(fused['label'] == 0).sum()} | 🔴 Fake: {(fused['label'] == 1).sum()}")


✅ Loaded Label: shape = (310866, 3)
✅ Loaded Motion: shape = (346788, 6)
✅ Loaded NMCS: shape = (804, 5)
✅ Loaded Muscle: shape = (804, 37)

✅ Video ID Overlap Stats:
🧩 Label ∩ Motion: 262
🧩 Label ∩ NMCS: 247
🧩 Label ∩ Muscle: 247
🧩 All 4 Intersecting: 247


MergeError: Passing 'suffixes' which cause duplicate columns {'label_x'} is not allowed.

In [4]:
import pandas as pd

# === File Paths ===
label_path = r"D:\DETECTION 1.0\Celeb-DF\aligned_frame_labels.csv"
motion_path = r"D:\DETECTION 1.0\Celeb-DF\combined_motion.csv"
nmcs_path = r"D:\DETECTION 1.0\Celeb-DF\nmcs_scores.csv"
muscle_path = r"D:\DETECTION 1.0\Celeb-DF\muscle_tension_features.csv"
output_path = r"D:\DETECTION 1.0\Celeb-DF\fused_features_new.csv"

# === Load CSVs ===
label_df = pd.read_csv(label_path)
motion_df = pd.read_csv(motion_path)
nmcs_df = pd.read_csv(nmcs_path)
muscle_df = pd.read_csv(muscle_path)

print(f"✅ Loaded Label: shape = {label_df.shape}")
print(f"✅ Loaded Motion: shape = {motion_df.shape}")
print(f"✅ Loaded NMCS: shape = {nmcs_df.shape}")
print(f"✅ Loaded Muscle: shape = {muscle_df.shape}")

# === Normalize video_id (keep only base video id like 00000) ===
label_df['video_id'] = label_df['video_id'].astype(str).apply(lambda x: x.split('_')[0])
motion_df['video_id'] = motion_df['video_id'].astype(str).apply(lambda x: x.split('_')[0])
nmcs_df['video_id'] = nmcs_df['video_id'].astype(str)
muscle_df['video_id'] = muscle_df['video_id'].astype(str)

# === Group frame-based data to video-level by averaging ===
label_grouped = label_df.groupby('video_id').mean(numeric_only=True).reset_index()
motion_grouped = motion_df.groupby('video_id').mean(numeric_only=True).reset_index()

# === Check intersections ===
label_ids = set(label_grouped['video_id'])
motion_ids = set(motion_grouped['video_id'])
nmcs_ids = set(nmcs_df['video_id'])
muscle_ids = set(muscle_df['video_id'])

intersect_all = label_ids & motion_ids & nmcs_ids & muscle_ids
print(f"\n✅ Video ID Overlap Stats:")
print(f"🧩 Label ∩ Motion: {len(label_ids & motion_ids)}")
print(f"🧩 Label ∩ NMCS: {len(label_ids & nmcs_ids)}")
print(f"🧩 Label ∩ Muscle: {len(label_ids & muscle_ids)}")
print(f"🧩 All 4 Intersecting: {len(intersect_all)}")

# === Filter all to common video_ids only ===
label_grouped = label_grouped[label_grouped['video_id'].isin(intersect_all)]
motion_grouped = motion_grouped[motion_grouped['video_id'].isin(intersect_all)]
nmcs_df = nmcs_df[nmcs_df['video_id'].isin(intersect_all)]
muscle_df = muscle_df[muscle_df['video_id'].isin(intersect_all)]

# === Safety check: ensure uniqueness ===
assert label_grouped['video_id'].is_unique
assert motion_grouped['video_id'].is_unique
assert nmcs_df['video_id'].is_unique
assert muscle_df['video_id'].is_unique

# === Drop duplicate 'label' column from non-primary sources ===
for df in [motion_grouped, nmcs_df, muscle_df]:
    if 'label' in df.columns:
        df.drop(columns=['label'], inplace=True)

# === Merge all ===
fused = label_grouped.merge(motion_grouped, on='video_id') \
                     .merge(nmcs_df, on='video_id') \
                     .merge(muscle_df, on='video_id')

# === Save output ===
fused.to_csv(output_path, index=False)
print(f"\n✅ Fusion complete. Saved to: {output_path}")
print(f"📦 Final shape: {fused.shape}")
print(f"🟢 Real: {(fused['label'] == 0).sum()} | 🔴 Fake: {(fused['label'] == 1).sum()}")


✅ Loaded Label: shape = (310866, 3)
✅ Loaded Motion: shape = (346788, 6)
✅ Loaded NMCS: shape = (804, 5)
✅ Loaded Muscle: shape = (804, 37)

✅ Video ID Overlap Stats:
🧩 Label ∩ Motion: 262
🧩 Label ∩ NMCS: 247
🧩 Label ∩ Muscle: 247
🧩 All 4 Intersecting: 247

✅ Fusion complete. Saved to: D:\DETECTION 1.0\Celeb-DF\fused_features_new.csv
📦 Final shape: (247, 43)
🟢 Real: 247 | 🔴 Fake: 0


In [5]:
label_df[label_df['label'] == 1]['video_id'].unique()


array(['id1', 'id3', 'id4', 'id6', 'id7', 'id8', 'id9'], dtype=object)

In [6]:
print("Fake in Motion:", motion_df[motion_df['label'] == 1]['video_id'].nunique())
print("Fake in NMCS:", nmcs_df[nmcs_df['label'] == 1]['video_id'].nunique() if 'label' in nmcs_df else "No label column")
print("Fake in Muscle:", muscle_df[muscle_df['label'] == 1]['video_id'].nunique() if 'label' in muscle_df else "No label column")


Fake in Motion: 7
Fake in NMCS: No label column
Fake in Muscle: No label column


In [7]:
# Check video_id formatting and labels for each source

# First ensure correct formatting of video_id
label_df['video_id'] = label_df['video_id'].astype(str).apply(lambda x: x.split('_')[0])
motion_df['video_id'] = motion_df['video_id'].astype(str).apply(lambda x: x.split('_')[0])
nmcs_df['video_id'] = nmcs_df['video_id'].astype(str)
muscle_df['video_id'] = muscle_df['video_id'].astype(str)

# Get unique fake video_ids
fake_label_ids = set(label_df[label_df['label'] == 1]['video_id'].unique())
fake_motion_ids = set(motion_df[motion_df['label'] == 1]['video_id'].unique()) if 'label' in motion_df.columns else set()
fake_nmcs_ids = set(nmcs_df[nmcs_df['label'] == 1]['video_id'].unique()) if 'label' in nmcs_df.columns else set()
fake_muscle_ids = set(muscle_df[muscle_df['label'] == 1]['video_id'].unique()) if 'label' in muscle_df.columns else set()

# Print how many fakes are in each
print(f"🧠 Fake Videos Found:")
print(f"🟣 In Label: {len(fake_label_ids)}")
print(f"🔵 In Motion: {len(fake_motion_ids)}")
print(f"🟢 In NMCS: {len(fake_nmcs_ids)}")
print(f"🟡 In Muscle: {len(fake_muscle_ids)}")


🧠 Fake Videos Found:
🟣 In Label: 7
🔵 In Motion: 7
🟢 In NMCS: 0
🟡 In Muscle: 0
